In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
import uuid

In [3]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp

print(f"Number of TPU cores available: {xm.xrt_world_size()}")
print(f"Current TPU ordinal: {xm.get_ordinal()}")

E0000 00:00:1747417818.089569    2990 common_lib.cc:621] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:232


Number of TPU cores available: 1
Current TPU ordinal: 0


In [4]:
# TPU-specific imports
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

In [5]:
# Data Loading and Preprocessing
base_path = '/kaggle/input/data'
image_dirs = [f'images_{str(i).zfill(3)}' for i in range(1, 13)]
data_entry_csv_path = os.path.join(base_path, 'Data_Entry_2017.csv')

# Load metadata
data_entry_df = pd.read_csv(data_entry_csv_path)
data_entry_df = data_entry_df.loc[:, ~data_entry_df.columns.str.contains('^Unnamed')]
print(f"Total data entries: {len(data_entry_df)}")

class SSLCustomDataset(Dataset):
    def __init__(self, df, image_dirs, base_path, target_size=(224, 224)):
        self.df = df
        self.image_dirs = image_dirs
        self.base_path = base_path
        self.target_size = target_size
        
        self.image_path_map = self._build_image_path_map()
        self.valid_indices = [i for i in range(len(df)) if self.image_path_map.get(df.iloc[i]['Image Index'])]
        print(f"Filtered dataset to {len(self.valid_indices)} valid images")

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomResizedCrop(target_size, scale=(0.5, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply([
                transforms.ColorJitter(0.1, 0.1, 0.1, 0.05)
            ], p=0.5),
            transforms.RandomGrayscale(p=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
    def _build_image_path_map(self):
        path_map = {}
        for dir_name in self.image_dirs:
            dir_path = os.path.join(self.base_path, dir_name, 'images')
            if os.path.exists(dir_path):
                for img_file in os.listdir(dir_path):
                    path_map[img_file] = os.path.join(dir_path, img_file)
        return path_map
    
    def __len__(self):
        return len(self.valid_indices)
    

    def __getitem__(self, idx):
        idx = self.valid_indices[idx]
        row = self.df.iloc[idx]
        img_path = self.image_path_map[row['Image Index']]
        
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError(f"Failed to load image: {img_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img1 = self.transform(img)
        img2 = self.transform(img)
        
        return img1, img2


Total data entries: 112120


In [6]:
# Model Definitions
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.resnet = models.resnet18(weights='IMAGENET1K_V1')
        self.resnet.fc = nn.Identity()

    def forward(self, x):
        return self.resnet(x)

class MLPHead(nn.Module):
    def __init__(self, in_dim=512, hidden_dim=512, out_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim, bias=False),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, out_dim, bias=False),
            nn.BatchNorm1d(out_dim, affine=False)
        )
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def forward(self, x):
        z = self.net(x)
        return F.normalize(z, dim=1)
        
class SSLModel(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        self.projector = MLPHead()

    def forward(self, x1, x2):
        h1 = self.encoder(x1)
        h2 = self.encoder(x2)
        z1 = self.projector(h1)
        z2 = self.projector(h2)
        return z1, z2
        
class NTXentLoss(nn.Module):
    def __init__(self, temperature=0.1):  
        super().__init__()
        self.temperature = temperature
        
    def forward(self, z1, z2):
        batch_size = z1.size(0)
        features = torch.cat([z1, z2], dim=0)
        similarity = torch.mm(features, features.t()) / self.temperature
        
        pos_sim = torch.diag(similarity, batch_size)
        if torch.rand(1).item() < 0.01:
            xm.master_print(f"Positive similarity: {pos_sim.mean().item():.4f}")
        
        mask = torch.eye(2 * batch_size, device=similarity.device)
        mask = 1 - mask
        similarity = similarity * mask
        
        labels = torch.zeros(2 * batch_size, device=similarity.device, dtype=torch.int64)
        labels[0:batch_size] = torch.arange(batch_size, 2*batch_size)
        labels[batch_size:2*batch_size] = torch.arange(0, batch_size)
        
        loss = F.cross_entropy(similarity, labels)
        return loss

In [13]:
# Training function adapted for TPU
def train():
    device = xm.xla_device()
    # Create dataset and dataloader
    batch_size = 128 * xm.xrt_world_size()  # Scale batch size with number of cores
    dataset = SSLCustomDataset(data_entry_df, image_dirs, base_path)
    sampler = torch.utils.data.distributed.DistributedSampler(
        dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True)
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=sampler,
        num_workers=4,
        drop_last=True)
    
    # Wrap with parallel loader
    device_loader = pl.MpDeviceLoader(dataloader, device)

    # Instantiate model
    model = SSLModel(Encoder()).to(device)
    criterion = NTXentLoss(temperature=0.1).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-3 * xm.xrt_world_size())  # Scale learning rate
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)
    scaler = GradScaler()

    num_epochs = 5
    accum_steps = 16
    print_freq = 200
    
    for epoch in range(num_epochs):
        model.train()
        sampler.set_epoch(epoch)
        total_loss = 0
        batch_count = 0
        
        for batch_idx, (img1, img2) in enumerate(tqdm(device_loader, 
                                                    desc=f"Epoch {epoch + 1}/{num_epochs}",
                                                    disable=not xm.is_master_ordinal())):
            with autocast():
                z1, z2 = model(img1, img2)
                loss = criterion(z1, z2) / accum_steps

            scaler.scale(loss).backward()
            
            if (batch_idx + 1) % print_freq == 0 and xm.is_master_ordinal():
                # Removed cosine similarity computation and printing
                xm.master_print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}, Loss: {loss.item() * accum_steps:.4f}")
            
            if (batch_idx + 1) % accum_steps == 0 or (batch_idx + 1) == len(dataloader):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                xm.mark_step()  # Important for TPU execution

            total_loss += loss.item() * accum_steps
            batch_count += 1

        scheduler.step()
        avg_loss = total_loss / batch_count
        xm.master_print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

        if xm.is_master_ordinal():
            torch.save(model.state_dict(), f"ssl_model_epoch_{epoch + 1}.pth")

    if xm.is_master_ordinal():
        final_save_path = 'ssl_model_final.pth'
        torch.save({
            'model_state_dict': model.state_dict(),
            'encoder_architecture': 'resnet18',
            'projector_dims': [512, 512, 128],
        }, final_save_path)
        print(f"Training complete! Model saved to {final_save_path}")


In [14]:
# Start training
def _mp_fn(rank, flags):
    # Initialize TPU inside the spawned process
    device = xm.xla_device()
    print(f"TPU {rank}: Device {device} initialized")
    train()
# Launch TPU training
FLAGS = {}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs= 1, start_method='fork')


TPU 0: Device xla:0 initialized
Filtered dataset to 112120 valid images


Epoch 1/5:   2%|▏         | 14/875 [00:20<08:35,  1.67it/s] 

Positive similarity: 3.2744


Epoch 1/5:   3%|▎         | 26/875 [00:38<27:17,  1.93s/it]

Positive similarity: 7.7182


...

Positive similarity: 9.5025


Epoch 5/5:  91%|█████████▏| 799/875 [15:26<00:35,  2.15it/s]

Epoch 5/5, Batch 800, Loss: 0.0416


Epoch 5/5: 100%|██████████| 875/875 [16:52<00:00,  1.16s/it]


Epoch 5/5, Average Loss: 0.0427
Training complete! Model saved to ssl_model_final.pth
